In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2021)
nltk.download('wordnet')
from gensim import corpora, models
from gensim.models import CoherenceModel
from __future__ import print_function
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
stemmer = SnowballStemmer("english")
def remove_symbols(val):
    if(len(val)==0):
        return val
    if(val[0]=='"'):
        val=val[1:]
    if(val[-1]==','):
        val=val[:-1]
    if(val[-1]=='"'):
        val=val[:-1]
    return val


def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess_sentence(text):
    #print('preprocessing ',text)
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [3]:
def preprocess(json_list, year):    
    index =  0
    tokens = []
    flag = 0
    for line in json_list:
        line = line.strip()
        if('"title"' in line):
            move = len('"title" : ')
            processed_token = preprocess_sentence(remove_symbols(line[move:]))
            for val in processed_token:
                tokens.append(val)
        if('"abstract"' in line):
            move = len('"abstract" : ')
            if(line[move:]!='"",'):
                processed_token = preprocess_sentence(remove_symbols(line[move:]))
                for val in processed_token:
                    tokens.append(val)
        if('"keywords"' in line):
            cur_index = index+1
            while(cur_index < len(json_list)):
                if(']' in json_list[cur_index]):
                    break
                if(len(json_list[cur_index])>1):    
                    tokens.append(remove_symbols(json_list[cur_index].strip()))
                cur_index+=1
        if('"year" :' in line):
            yr = line.strip()
            move = len('"year" : NumberInt(')
            yr = yr[move:]
            yr = yr[:-2]
            for i in range(year, year+20):
                if(yr == str(i)):
                    flag=1
                    break
        if('"fos" : [' in line):
            cur_index = index+1
            while(cur_index < len(json_list)):
                if(']' in json_list[cur_index]):
                    break
                if(len(json_list[cur_index].strip())==0):
                    cur_index+=1
                    continue
                fos = remove_symbols(json_list[cur_index].strip())
                tokens.append(fos)
                cur_index+=1
        index+=1
    if(flag==1):
        return tokens
    else:
        return []




def read_file(file, year):
    processed_docs = []
    stack = []
    cnt=0
    ct=0
    
    while(True):
        line = file.readline()
        if(not line):
            break
        if('"title"' in line):  #end of json object
            cnt+=1
            if(cnt>1):
                #print(cnt)
                tokens = preprocess(stack, year)
                stack.clear()
                if(len(tokens)>0):
                    processed_docs.append(tokens)
                if((cnt%100000)==0):
                    print(cnt)
                    cnt=2
               
                
             
        stack.append(line)
        
    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=8, no_above=0.60,keep_n=400)
    print('dictionary items after preprocessing:')
    for k, v in dictionary.iteritems():
        print(k, v)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=dictionary, passes=2, workers=4)
    print('year {} to {}'.format(year, year+20))
    print('********************************************************')
    for idx, topic in lda_model_tfidf.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))
    
    print('********************************************************')
    print('******************Coherence score is********************')
    coherence_model_lda = CoherenceModel(model = lda_model_tfidf, texts = processed_docs, dictionary=dictionary, coherence='c_v')
    print(coherence_model_lda.get_coherence())
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
    
    
    return viz
     

In [4]:
viz_list = []
for i in range(1981, 2022, 20):
    file = open('dblpv13.json',  encoding="utf8")
    viz_list.append(read_file(file, i))
    file.close()
    

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
dictionary items after preprocessing:
0 Artificial intelligence
1 Computer science
2 Pattern recognition
3 computer science
4 problem
5 shape
6 solution
7 access
8 allow
9 analysis
10 article
11 best
12 concept
13 construct
14 design
15 different
16 evaluation
17 examine
18 group
19 important
20 issue
21 level
22 performance
23 process
24 quality
25 test
26 understand
27 value
28 components
29 consider
30 control
31 correspond
32 develop
33 distribute
34 example
35 focus
36 functional
37 investigate
38 language
39 method
40 present
41 program
42 prove
43 rule
44 simulation
45 specification
46 step
47 systems
48 work
49 al


KeyboardInterrupt



In [ ]:
viz_list[0]

In [ ]:
viz_list[1]

In [8]:
viz_list[0]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.125928 -0.042992       1        1  20.882458
0     -0.113663 -0.061949       2        1  16.147225
3     -0.133611  0.034642       3        1  15.355876
6      0.031495 -0.073668       4        1  14.519201
5      0.126866  0.076320       5        1  12.432355
2      0.013013 -0.015495       6        1  12.349449
1     -0.050028  0.083140       7        1   8.313435, topic_info=                        Term         Freq        Total Category  logprob  \
267     Programming language  1211.000000  1211.000000  Default  30.0000   
253         Computer science  3212.000000  3212.000000  Default  29.0000   
255              Mathematics  2647.000000  2647.000000  Default  28.0000   
362     Discrete mathematics  1670.000000  1670.000000  Default  27.0000   
368  Artificial intelligence  1148.000000  1148.000000  Default  26.0000   
..                       ...          ...          ...      ...      ...   
193                   number   102.677001   931.134138   Topic7  -4.9720   
37                  problems   102.466362   934.746613   Topic7  -4.9741   
109                  process   106.925976  1278.726491   Topic7  -4.9315   
242                  systems   110.550137  1714.605273   Topic7  -4.8981   
132                     data   107.487467  1632.587395   Topic7  -4.9262   

     loglift  
267  30.0000  
253  29.0000  
255  28.0000  
362  27.0000  
368  26.0000  
..       ...  
193   0.2825  
37    0.2766  
109   0.0058  
242  -0.2542  
132  -0.2332  

[334 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
388       1  0.002340  Algebra
388       2  0.057335  Algebra
388       3  0.879921  Algebra
388       4  0.011701  Algebra
388       5  0.005851  Algebra
...     ...       ...      ...
57        3  0.022985    years
57        4  0.160895    years
57        5  0.101134    years
57        6  0.068955    years
57        7  0.018388    years

[1638 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 4, 7, 6, 3, 2])